In [1]:
import pygame as pg
from pygame.math import Vector2
import sys
from time import sleep
from random import randint

pygame 2.1.2 (SDL 2.0.18, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pg.init()

(5, 0)

In [3]:
fps = 24
win_w = 600
win_h = 500

game_w = 24
game_h = 20
bombs = 99
coords = [-1, 1]

box_colour = 250, 90, 90
mbox_colour = 235, 235, 235

In [4]:
screen = pg.display.set_mode((win_w, win_h))
pg.display.set_caption("Minesweeper")

clock = pg.time.Clock()

In [5]:
def draw_text(surface, text, size, x, y):
    # font_name = pg.font.match_font("Apple Color Emoji")
    font = pg.font.Font(None, size)
    text_surface = font.render(text, True, (30, 30, 30))
    text_rect = text_surface.get_rect()
    text_rect.center = (x, y)
    surface.blit(text_surface, text_rect)

In [6]:
def map_gen(WIDTH, HEIGHT, bomb_rem):
    
    map = []
    row = []
    
    box_rem = WIDTH*HEIGHT
    
    # Random generation of bomb
    for h in range(HEIGHT):
        for w in range(WIDTH):
            
            if randint(1, box_rem) <= bomb_rem:
                row.append("\U0001f4a3")
                bomb_rem -= 1
            else:
                row.append(0)
        
            box_rem -= 1
        
        map.append(row)
        row = []
    
    # Number generation
    for i_row, row in enumerate(map):
        for i_box, n in enumerate(row):
            if n == "\U0001f4a3":
                for coord in coords:
                    try:
                        if i_box+coord >= 0:
                            row[i_box+coord] += 1
                    except:
                        pass
                    
                for index in range(i_box-1, i_box+2):
                    if index >= 0 and i_row-1 >= 0:
                        try:
                            map[i_row-1][index] += 1
                        except:
                            pass
                    
                for index in range(i_box-1, i_box+2):
                    if index >= 0:
                        try:
                            map[i_row+1][index] += 1
                        except:
                            pass
                    
    return map

In [7]:
class Box(pg.sprite.Sprite):
    def __init__(self, offset_x, offset_y, colour, text):
        super().__init__()
        self.image = pg.Surface((25, 25))
        self.image.fill(colour)
        self.colour = colour
        self.rect = self.image.get_rect()
        self.offset_x = offset_x
        self.offset_y = offset_y
        self.opened = 0
        if text == "0":
            self.text = " "
        else:
            self.text = text
        
    def update(self):
        
        if self.opened == 1:
            
            if self.colour == (250, 90, 90):
                self.image.fill((230, 230, 230))
            else:
                self.image.fill((250, 250, 250))
            
            draw_text(self.image, self.text, 13, 12.5, 12.5)
                
        self.rect = self.image.get_rect(topleft = Vector2(self.offset_x, self.offset_y))

In [8]:
class MapBox(pg.sprite.Sprite):
    def __init__(self, offset_x, offset_y, colour, text):
        super().__init__()
        self.image = pg.Surface((25, 25))
        self.image.fill(colour)
        self.rect = self.image.get_rect()
        self.offset_x = offset_x
        self.offset_y = offset_y
        if text == "0":
            self.text = " "
        else:
            self.text = text
        
    def update(self):
        self.rect = self.image.get_rect(topleft = Vector2(self.offset_x, self.offset_y))
        draw_text(self.image, self.text, 12, 12.5, 12.5)

In [9]:
mboxes = []
boxes = []

map = map_gen(game_w, game_h, bombs)
for x in map: print(x)

row = 0
column = 0

[1, '💣', 3, '💣', 4, '💣', 1, 0, 1, '💣', 1, 0, 1, 1, 1, 0, 0, 1, 2, 2, 2, 2, '💣', 1]
[1, 1, 4, '💣', '💣', 2, 2, 1, 3, 2, 2, 0, 1, '💣', 1, 0, 1, 2, '💣', '💣', 4, '💣', 3, 1]
[1, 2, 4, '💣', 5, 2, 1, '💣', 3, '💣', 2, 2, 2, 2, 1, 0, 1, '💣', 3, 3, '💣', '💣', 3, 1]
['💣', 4, '💣', '💣', '💣', 2, 2, 2, '💣', 3, '💣', 3, '💣', 1, 1, 2, 4, 3, 2, 1, 2, 3, 4, '💣']
[2, '💣', '💣', 4, 3, '💣', 1, 1, 1, 2, 2, '💣', 2, 2, 2, '💣', '💣', '💣', 1, 0, 0, 1, '💣', '💣']
[2, 3, 2, 1, 1, 2, 2, 1, 0, 1, 2, 2, 1, 1, '💣', 3, 3, 2, 1, 0, 0, 2, 4, 4]
['💣', 1, 0, 0, 0, 1, '💣', 1, 1, 2, '💣', 1, 0, 1, 1, 1, 1, 2, 3, 2, 1, 1, '💣', '💣']
[2, 2, 0, 0, 1, 2, 2, 1, 2, '💣', 3, 1, 0, 0, 0, 0, 1, '💣', '💣', '💣', 2, 1, 2, 2]
['💣', 2, 1, 1, 1, '💣', 1, 0, 2, '💣', 2, 1, 1, 1, 0, 0, 1, 2, 4, '💣', 3, 1, 1, 1]
[1, 2, '💣', 1, 1, 1, 1, 0, 1, 1, 1, 1, '💣', 1, 1, 1, 1, 1, 3, 4, '💣', 1, 1, '💣']
[0, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, '💣', 1, 2, '💣', '💣', 2, 1, 2, 2]
[0, 1, '💣', 1, 1, 2, '💣', 1, 1, '💣', 1, 2, '💣', 2, 2, 2, 3, 3, '💣', 3, 1, 0, 1, '💣']
[1

In [10]:
for x in range(480):
    box = Box(column*25, row*25, box_colour, str(map[row][column]))
    
    column += 1
    if column == 24:
        row += 1
        column = 0
        boxes.append(box)
        continue
    
    # if mbox_colour == (235, 235, 235):
    #     mbox_colour = (250, 250, 250)
    # else:
    #     mbox_colour = (235, 235, 235)
        
    if box_colour == (250, 90, 90):
        box_colour = (250, 125, 125)
    else:
        box_colour = (250, 90, 90)

    # mboxes.append(mbox)
    boxes.append(box)

In [11]:
# mbox_group = pg.sprite.Group()
# for mbox in mboxes:
#     mbox_group.add(mbox)
    
box_group = pg.sprite.Group()
for box in boxes:
    box_group.add(box)

In [12]:
valid_map = 0

In [ ]:
while True:
    global map
    screen.fill("black")

    keys = pg.key.get_pressed()
    if keys[pg.K_ESCAPE]:
        pg.quit()
        exit()
    
    for event in pg.event.get():
        if event.type == pg.QUIT: 
            pg.quit()
            exit()
        if event.type == pg.MOUSEBUTTONUP:
            for box in boxes:
                if box.rect.collidepoint(pg.mouse.get_pos()):
                    box.opened = 1
                    # offset_x, offset_y, index = box.offset_x, box.offset_y, box.index
                    # if box.colour == (250, 90, 90):
                    #     colour = mbox_colour = (250, 250, 250)
                    # else: colour = (235, 235, 235)
                    # box = MapBox(offset_x, offset_y, colour, map[2])
        
    for box in boxes:
        if box.rect.collidepoint(pg.mouse.get_pos()) and box.opened == 0:
            box.image.fill((255, 200, 200))
        else:
            box.image.fill(box.colour)
    
    # mbox_group.update()
    # mbox_group.draw(screen)
    
    box_group.update()
    box_group.draw(screen)
        
    clock.tick(fps) 
    pg.display.flip()
